## D. Kinney DSC 550 9.3 Exercise: Neural Network Classifiers

1. **Neural Network Classifier with Scikit**

Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using scikit-learn. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.

2. **Neural Network Classifier with Keras**

Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using Keras. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.

3. **Classifying Images**

In chapter 20 of the Machine Learning with Python Cookbook, implement the code found in section 20.15 classify MSINT images using a convolutional neural network. Report the accuracy of your results.

*********************************************
#### 1. Neural Network Classifier with Scikit

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/categorized-comments.csv')
df.dropna(inplace=True)
df.shape

(606467, 3)

In [3]:
# This dataset is HUGE. Sample 50k observations...
df = df.sample(50000)

In [4]:
df.shape

(50000, 3)

In [19]:
df.cat.value_counts()

video_games               35907
sports                    12051
science_and_technology     2042
Name: cat, dtype: int64

In [5]:
# Extracting features from text files
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.txt)
X_train_counts.shape

(50000, 40312)

In [6]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(50000, 40312)

In [8]:
# Performance of MLP Classifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.txt, df.cat, test_size=0.33, random_state=42)

# Consolidate steps into a pipeline...
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

text_clf = Pipeline([
    ('vect', CountVectorizer()), 
    ('tfidf', TfidfTransformer()), 
    ('ann', MLPClassifier(hidden_layer_sizes=[30,30],
                          max_iter=50, 
                          verbose = True))]) 
scoring = 'f1_micro'
scores = cross_val_score(text_clf, X_train, y_train, scoring=scoring)
text_clf = text_clf.fit(X_train, y_train)

Iteration 1, loss = 0.79446124
Iteration 2, loss = 0.49173478
Iteration 3, loss = 0.33787678
Iteration 4, loss = 0.24978728
Iteration 5, loss = 0.19366401
Iteration 6, loss = 0.16098453
Iteration 7, loss = 0.13905346
Iteration 8, loss = 0.12439685
Iteration 9, loss = 0.11334202
Iteration 10, loss = 0.10444609
Iteration 11, loss = 0.09775084
Iteration 12, loss = 0.09112227
Iteration 13, loss = 0.08522626
Iteration 14, loss = 0.08111368
Iteration 15, loss = 0.07752989
Iteration 16, loss = 0.07493085
Iteration 17, loss = 0.07181110
Iteration 18, loss = 0.06996176
Iteration 19, loss = 0.06777535
Iteration 20, loss = 0.06585833
Iteration 21, loss = 0.06449156
Iteration 22, loss = 0.06370003
Iteration 23, loss = 0.06169329
Iteration 24, loss = 0.06087726
Iteration 25, loss = 0.06074255
Iteration 26, loss = 0.05992138
Iteration 27, loss = 0.05899985
Iteration 28, loss = 0.05824092
Iteration 29, loss = 0.05697465
Iteration 30, loss = 0.05701648
Iteration 31, loss = 0.05650450
Iteration 32, los

In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import classification_report

print(f"f1 scores: {scores}")
predicted = text_clf.predict(X_test)
print("Accuracy: ", np.mean(predicted == y_test))

print("Confusion Matrix:\n", confusion_matrix(y_test, predicted))
print("Classification Report:\n", classification_report(y_test,predicted))
print("Accuracy: ", accuracy_score(y_test,predicted))

f1 scores: [0.78656716 0.78626866 0.79477612 0.79119403 0.78597015]
Accuracy:  0.7901212121212121
Confusion Matrix:
 [[  274    45   343]
 [   53  2206  1740]
 [  192  1090 10557]]
Classification Report:
                         precision    recall  f1-score   support

science_and_technology       0.53      0.41      0.46       662
                sports       0.66      0.55      0.60      3999
           video_games       0.84      0.89      0.86     11839

              accuracy                           0.79     16500
             macro avg       0.67      0.62      0.64     16500
          weighted avg       0.78      0.79      0.78     16500

Accuracy:  0.7901212121212121


*********************************************
#### 2. Neural Network Classifier with Keras

In [31]:
from keras.layers import Dense 
from keras.models import Sequential 

N_FEATURES = 32791
N_CLASSES = 3

def build_network():
    """ 
    Create a function that returns a compiled neural network 
    """ 
    nn = Sequential() 
    nn.add( Dense(30, activation ='relu', input_shape =( N_FEATURES,))) 
    nn.add( Dense(30, activation ='relu')) 
    nn.add( Dense(N_CLASSES, activation ='softmax')) 
    nn.compile( 
        loss ='categorical_crossentropy', 
        optimizer ='adam', 
        metrics =['accuracy'] ) 
    return nn

In [34]:
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer 

keras_clf = Pipeline([ 
    ('tfid', TfidfVectorizer( max_features = N_FEATURES)), 
    ('nn', KerasClassifier( build_fn = build_network, 
                            epochs = 20, 
                            batch_size = 128))
])

scores = cross_val_score(keras_clf, X_train, y_train, scoring ='accuracy', n_jobs =-1) 
keras_clf.fit(X_train, y_train) 
print(scores)
keras_predicted = keras_clf.predict(X_test)
print(np.mean(keras_predicted == y_test))

Epoch 1/20
33500/33500 [==============================] - 13s 395us/step - loss: 0.6339 - accuracy: 0.7582
Epoch 2/20
33500/33500 [==============================] - 13s 394us/step - loss: 0.3422 - accuracy: 0.8666
Epoch 3/20
33500/33500 [==============================] - 13s 390us/step - loss: 0.2373 - accuracy: 0.9097
Epoch 4/20
33500/33500 [==============================] - 13s 400us/step - loss: 0.1874 - accuracy: 0.9289
Epoch 5/20
33500/33500 [==============================] - 13s 398us/step - loss: 0.1589 - accuracy: 0.9396
Epoch 6/20
33500/33500 [==============================] - 13s 398us/step - loss: 0.1394 - accuracy: 0.9474
Epoch 7/20
33500/33500 [==============================] - 13s 397us/step - loss: 0.1240 - accuracy: 0.9528
Epoch 8/20
33500/33500 [==============================] - 14s 404us/step - loss: 0.1106 - accuracy: 0.9575
Epoch 9/20
33500/33500 [==============================] - 14s 403us/step - loss: 0.1002 - accuracy: 0.9618
Epoch 10/20
33500/33500 [============

In [35]:
print("Confusion Matrix:\n", confusion_matrix(y_test, keras_predicted))
print("Classification Report:\n", classification_report(y_test,keras_predicted))
print("Accuracy: ", accuracy_score(y_test,keras_predicted))

Confusion Matrix:
 [[  278    51   333]
 [   44  2410  1545]
 [  155  1175 10509]]
Classification Report:
                         precision    recall  f1-score   support

science_and_technology       0.58      0.42      0.49       662
                sports       0.66      0.60      0.63      3999
           video_games       0.85      0.89      0.87     11839

              accuracy                           0.80     16500
             macro avg       0.70      0.64      0.66     16500
          weighted avg       0.79      0.80      0.80     16500

Accuracy:  0.7998181818181819


*********************************************
#### 3. Classifying Images

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

# Set that the color channel value will be first
K.set_image_data_format("channels_first")

# Set seed
np.random.seed(0)

# Set image information
channels = 1
height = 28
width = 28

# Load data and target from MNIST data
(data_train, target_train), (data_test, target_test) = mnist.load_data()

# Reshape training image data into features
data_train = data_train.reshape(data_train.shape[0], channels, height, width)

# Reshape test image data into features
data_test = data_test.reshape(data_test.shape[0], channels, height, width)

# Rescale pixel intensity to between 0 and 1
features_train = data_train / 255
features_test = data_test / 255

# One-hot encode target
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

# Start neural network
network = Sequential()

# Add convolutional layer with 64 filters, a 5x5 window, and ReLU activation function
network.add(Conv2D(filters=64,
                   kernel_size=(5, 5),
                   input_shape=(channels, width, height),
                   activation='relu'))

# Add max pooling layer with a 2x2 window
network.add(MaxPooling2D(pool_size=(2, 2)))

# Add dropout layer
network.add(Dropout(0.5))

# Add layer to flatten input
network.add(Flatten())

# # Add fully connected layer of 128 units with a ReLU activation function
network.add(Dense(128, activation="relu"))

# Add dropout layer
network.add(Dropout(0.5))

# Add fully connected layer with a softmax activation function
network.add(Dense(number_of_classes, activation="softmax"))

# Compile neural network
network.compile(loss="categorical_crossentropy", # Cross-entropy
                optimizer="rmsprop", # Root Mean Square Propagation
                metrics=["accuracy"]) # Accuracy performance metric

# Train neural network
network.fit(features_train, # Features
            target_train, # Target
            epochs=2, # Number of epochs
            verbose=0, # Don't print description after each epoch

batch_size=1000, # Number of observations per batch
            validation_data=(features_test, target_test)) # Data for evaluation

In [ ]:
score = network.evaluate(features_test, target_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

*************************
**Additional References**

Machine Learning, NLP: Text Classification using scikit-learn, python and NLTK. Jul 23, 2017
https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a